This script loads pasted SMILES and inserts random smiles from the top-x of a dataset. The script creates A list of lists of lists where the first list contains the 30 initial startings lists of molecules for each run with 1 molecule nudged. The second list contains 30 initial starting lists with 2 molecules nudged and so on.

In [32]:
from MDRMF.dataset import Dataset
import random
import numpy as np

In [33]:
dataset = Dataset.load('../datasets/DTP_MQN_shuffled.pkl')
dataset

<Dataset X.shape: (131970, 42), y.shape: (131970,), w.shape: (131970,), ids: ['CCC/C=C(/C=[NH+]\\O)CC' 'CC[NH+](CC)Cc1ccccc1COC(=O)c1cccc(Cl)c1'
 'CCOC(=O)NC1CC1' ... 'CCOC(=O)N(CC[NH+](C)C)c1ccccc1'
 'COc1ccc(-c2cc(=O)c3c([O-])cc(C)cc3o2)cc1OC'
 'O=P(c1ccco1)(c1ccco1)c1ccco1']>

In [34]:
# List of SMILESs used in previous experiments
SMILES = [
    [
        "O=C(CCl)NCc1cnccn1",
        "Cn1cnc([N+](=O)[O-])c1SSc1c([N+](=O)[O-])ncn1C",
        "Cc1ccc(S(=O)(=O)N2Cc3ccc([N+](=O)[O-])cc3C2)cc1",
        "NC(=O)/N=N/c1c(F)c(F)c(F)c(F)c1F",
        "CCCOC(=O)NCCSC(=O)OCCC",
        "CC(=O)NC(CCC(=O)c1ccc(Cl)cc1)(C(=O)[O-])C(=O)[O-]",
        "CC[C@H](C)[C@H](N)C(=O)NCC(=O)OC",
        "C/C(=[NH+]\\S(=O)(=O)c1cccc([N+](=O)[O-])c1)OCC(C)C",
        "COc1ccc(/C=N/n2c(-c3ccccc3)n[nH]c2=O)c(OC)c1",
        "CC(=O)Nc1ccc(S(=O)(=O)F)cc1Cl"
    ],
    [
        "COc1c(O)cc(Br)c(C=O)c1N",
        "CCCCN1C(=O)NC(=O)[C@@]1(C)c1ccccc1",
        "c1ccc(C2=[NH+]N[C@@H]3c4ccccc4-c4ccccc4N3C2)cc1",
        "CCCCCC1(C(C)C)C(=O)NC(=O)NC1=O",
        "c1c2c(cc3c1OCN(C1CC1)C3)OCN(C1CC1)C2",
        "CCCCc1cc(O)ccc1O",
        "COc1ccc2cc(N3CCCC3)ccc2c1",
        "O=C(CSc1ncc[nH]1)c1ccc(Br)cc1",
        "C/C=C/[C@H](C(=O)NC(N)=O)C(C)C",
        "C=CCNC(=S)NCc1cccc2ccccc12"
    ],
    [
        "CCN(CC)/[NH+]=C/C[C@H](C)Nc1ccnc2cc(Cl)ccc12",
        "COc1ccc2nc(C)c(COC(C)=O)cc2c1",
        "C[NH+]1C=c2c3c(ccc2=C2C=C[C@H]4C=C5OCOC5=CC4=C21)OCO3",
        "O=C([O-])c1cccc(O)c1O",
        "O=CNc1cc(Br)cc2c1[nH]c(=O)c1ccccc12",
        "CCO/C=C1\\SC(=S)N(c2ccc(C(=O)[O-])cc2)C1=O",
        "CC(C)(C)C[NH+]1CCCN(c2ccc(Cl)cc2)C1",
        "O=C1c2ccccc2C(=O)N1Cc1cc([N+](=O)[O-])ccc1Cl",
        "COc1cccc(P(c2cccc(OC)c2)c2cccc(OC)c2)c1",
        "Cc1cccn2ncnc12"
    ],
    [
        "CC(=O)NC1=C([NH3+])C(=O)c2ccccc2C1=O",
        "O=C1C=C(c2ccccc2)O[C@H]1[C@@H]1Sc2ccccc2NC1=O",
        "O=C([O-])[C@H](c1ccccc1)c1cccc([C@@H](C(=O)[O-])c2ccccc2)n1",
        "COc1ccc(CC2=[N+](C)CCc3cc(OC)c(OC)cc32)cc1OC",
        "OCCCCC[NH2+]C1CCCCC1",
        "C1=Nc2c(ncnc2SCc2ccccc2)NN1",
        "CC1(C)O[C@@H]1C(=O)[O-]",
        "OC[C@H]1COc2cc(Cl)c(Cl)cc2O1",
        "c1nc2c3c4c(sc3ncn2n1)CCCC4",
        "COc1ccc(CCc2nccn2C)cc1OC"
    ],
    [
        "COc1ccc(Cl)c(S(N)(=O)=O)c1",
        "Cc1cc(SCc2ccc([N+](=O)[O-])cc2)nc(N)n1",
        "CCOC(=O)/C=C/c1ccc(N)cc1",
        "O=c1[nH]nc2c3cccc4cccc(c5ccc([O-])c1c52)c43",
        "CN(C=O)c1cc([N+](=O)[O-])ccc1OCc1ccccc1",
        "O=C(N/[NH+]=C\\C(F)(F)F)c1ccncc1",
        "CC(C)(CO)CNc1nc(N)nc(Cl)c1N",
        "CCOC(=O)N[C@H]1[C@@H]2OC(C)(C)O[C@@H]2O[C@H]1CC(=O)OC",
        "CCOC(=O)[C@H]1Cc2c(cc3c4c(cccc24)N(C(C)=O)C3)N(C)C1",
        "O=C(Nc1ccccc1CO)c1ccccc1"
    ],
    [
        "CS(=O)(=O)Oc1ccc([N+](=O)[O-])cc1",
        "CCCC1(C)[C@@H](C#N)C(=O)NC(=O)[C@@H]1C#N",
        "O=[N+]([O-])c1cccc([C@@H]2CC(c3ccccc3)=[NH+]N2c2ccccc2)c1",
        "O=S(=O)([O-])CC[C@@H](O)S(=O)(=O)[O-]",
        "Cc1c([N+](=O)[O-])ccc2cccnc12",
        "CCn1c(=O)c2c(ncn2CC)n(CC)c1=O",
        "CCn1c(Cl)c(/C=[NH+]\\Nc2ccc([N+](=O)[O-])cc2)c2ccccc21",
        "N#CS/C=C\\C(N)=O",
        "O=c1oc2ccccc2cc1-n1nnc2cc([N+](=O)[O-])ccc21",
        "CCOc1ccc(/N=C/c2cccc([N+](=O)[O-])c2)c(O)c1"
    ],
    [
        "CCn1c(=O)c(C#N)c2n(CC)c3cc(C)c(C)cc3n2c1=O",
        "O=C1c2ccccc2C(=O)c2c(OC[C@H]3CO3)ccc(OC[C@@H]3CO3)c21",
        "NC(=O)c1cccc(NC=O)c1",
        "CC(=O)c1ccccc1Br",
        "CC(=O)Nc1nnc(SC[NH+](C)C)s1",
        "C=C(Br)C[NH2+]CCCCCC",
        "CCOC(=O)c1[nH]c(CO)c(C(=O)OCC)c1C",
        "C[C@H]1CN(c2ccncc2[N+](=O)[O-])CCN1c1ccc(Cl)cc1",
        "CCC[C@H]1OCCN1CC",
        "CC1([N+](=O)[O-])COP(=O)(NCc2cccnc2)OC1"
    ],
    [
        "CCC(=O)Oc1cccc(OC(=O)CC)c1",
        "COc1cc(NC/C(C)=[NH+]/NC(N)=O)c2ncccc2c1",
        "Nc1nc(Cl)c2c(n1)CCC2",
        "Nc1ncnc2c1ncn2-c1ccccc1Cl",
        "COc1ccc(-c2nnc(-c3c(C)n(C)n(-c4ccccc4)c3=O)o2)cc1",
        "COc1cc2cccc(O)c2cc1OC",
        "O=C(NNC(=O)c1cccc2ccccc12)c1ccc(F)cc1",
        "CCOC(=O)[C@@H](C/[N+]([O-])=C/c1ccccc1)NC(C)=O",
        "COc1cc2c3c(c4cc(OC)c(OC)cc4c2cc1OC)C[NH+]1CCC[C@H]1C3",
        "CCN(CC)C(=O)Nc1cccc(O)c1"
    ],
    [
        "CNC(=O)NN",
        "C=C(C)[C@@H]1C[C@H](O)Cc2cc(F)ccc2N1",
        "C[NH+]1C(C)(C)CC(O)(C(=O)[O-])CC1(C)C",
        "Nc1ncc2[nH]c(=O)[nH]c2n1",
        "Cn1cc(C(=O)c2ccccc2)cc(C(=O)c2ccccc2)c1=O",
        "CCCCCCC[C@@H]1OCC[C@H](C)O1",
        "CCC(=[NH+]\\NC(=S)NC)/C(C)=[NH+]\\NC(=S)NC",
        "Nc1ccc(CCOc2ccc(N)cc2)cc1",
        "CCOc1ccc(N)c(SCC(=O)[O-])c1",
        "S=C1Nc2ccccc2CS1"
    ],
    [
        "CCOc1nc(C)cc(C(=O)[O-])c1C#N",
        "COc1ccccc1/C=C(/C)C(C)=O",
        "COC(=O)N1[C@@H](N2CCCC2=O)C(C)(C)S1(=O)=O",
        "CCOC(=O)CCN1C(=O)CC[C@H]1C(=O)OCC",
        "NC(=O)N/[NH+]=C\\c1ccc(O)cn1",
        "C1CSC(N2CCOCC2)=N1",
        "CCO[N+]([O-])=C1C=C(C(C)(C)C)C(=O)C(C(C)(C)C)=C1",
        "COc1cccc([C@@H]2CCCC[C@@H]2O)c1",
        "CCCn1c(N)c(Cl)c(=O)n(CC)c1=O",
        "COC(=O)/C=C/CC(=O)NS(C)(=O)=O"
    ],
    [
        "C[C@H](CC(=O)c1ccc(F)cc1)[NH+]1CCCCC1",
        "CSCC[C@H]([NH3+])[P@](C)(=O)[O-]",
        "N#C/C(=C\\Nc1ccccn1)C(=O)C1CC1",
        "C(=[NH+]/Nc1nncn2cccc12)\\c1ccccc1",
        "C[C@@]1(CCCOc2cccc(N)c2)[NH+]=C(N)[NH+]=C(N)N1c1ccccc1",
        "CC1(C(Cl)(Cl)Cl)C=CC(=[NH+]OCC(=O)[O-])C=C1",
        "C[NH+]1CCN(c2ccc3[nH]c(-c4c5cccccc-5[nH]c4=O)nc3c2)CC1",
        "CCO/[NH+]=C(\\C)C(=O)NNS(=O)(=O)c1ccc(C)cc1",
        "CCOC(=O)[C@@H](CC(C)C)NC(=O)c1cc2ccccc2[n+](C)c1",
        "Cc1ccc(/C=[NH+]/NC(N)=[NH2+])cc1"
    ],
    [
        "COc1cccc2c1CC[C@H]1C(C)(C)C(=O)CC[C@]21CC(=O)[O-]",
        "O=C(CCl)Nc1nncs1",
        "C[S@@](=O)CCC[NH+]=C=S",
        "CS(=O)(=O)Oc1ccccc1C(=O)[O-]",
        "COc1ccc(C[C@H](CO)NC(=O)c2ccccc2)cc1",
        "C[C@H](NC(=O)NCCCl)c1ccsc1",
        "CC[C@H](Cc1ccco1)C(=O)[O-]",
        "CC(C)[C@H](C)C1=CC(=O)C=CC1=O",
        "S=C(N/[NH+]=C/c1c[nH]c2ccccc12)NC12CC3CC(CC(C3)C1)C2",
        "NC(=O)c1ccc[n+](Cc2ccccc2)c1"
    ],
    [
        "O=C1N[C@@](O)(c2ccccc2)c2ccccc21",
        "COc1cc(O)c(C(=O)[C@@H]2Cc3ccccc32)c(OC)c1",
        "O=C(c1ccccc1)N1[NH+]=CCC[C@@H]1O",
        "CCOC(=O)[C@@H](C)Sc1n[nH]c(=O)[nH]c1=O",
        "O=C([O-])c1nc2ccccc2c2ncccc12",
        "COC(=O)[C@H]1Cc2cncn2C(=O)N1",
        "CCOC(=O)N/[NH+]=C/c1ccc(OC)c(O)c1",
        "O=c1cccc2n1Cc1cc3ccccc3nc1-2",
        "CN(C)/C=N/c1ccc(F)cc1",
        "CCOC(=O)C(Cc1cc2c(n1C)CCCC2)(NC(C)=O)C(=O)OCC"
    ],
    [
        "CCCCC=C1C(=O)NC(=S)NC1=O",
        "CC1(C)CCC[NH+]1CCOC(=O)[C@H](c1ccccc1)[C@H]1C=CCC1",
        "CCCCOC(=O)[C@@H]([NH3+])Cc1ccc(OC)c(Cl)c1",
        "CC(=O)Oc1cccnc1",
        "Cc1nnc2c(S)nccn12",
        "CCCC(=O)Nc1ccc(C)cc1",
        "CN1[NH+]=C2c3ccccc3CC[C@H]2[C@@H]1c1cccc(O)c1",
        "Cc1ccc(NC(=S)NC(=O)COc2ccccc2)cc1C",
        "CCOC(=O)NC(=O)CBr",
        "CCO/C=[NH+]\\C1=C(C#N)SC2=N/C(=C\\c3ccco3)C(=O)N21"
    ],
    [
        "Fc1cc(Cl)ccc1Cl",
        "CCOC(=O)/C(=C\\c1ccc(Cl)cc1)C(C)=O",
        "Cc1cccc(CNc2cccc(C)n2)n1",
        "N#CCN1CCCN(CC#N)C1",
        "CC(C)(C)[Si](C)(C)OC[C@]12CC[C@H]3C[NH2+][C@H]([C@H]31)[C@H](c1ccccc1)N2",
        "C[C@H](Cc1ccccc1)NC(N)=O",
        "CCn1cc(C(=O)[O-])c(=O)c2ccc(C)[n+](C)c21",
        "CC(=O)C(=Cc1cccc([N+](=O)[O-])c1)C(C)=O",
        "CC(=O)Oc1cccc2c1[nH]c(=O)c1c(OC(C)=O)c3c(cc12)OCO3",
        "CCc1nn2c(=O)nc(SC)nc2s1"
    ],
    [
        "N/C(=N/c1ccc(O)cc1)S(=O)(=O)[O-]",
        "CC(=O)Nc1nc(-c2nnc(S)n2-c2ccc(Cl)cc2)cs1",
        "COc1ccc(C(=O)NCC[NH+](C)C)c2nc3ccccc3cc12",
        "CC(=O)Nc1ccc(S(=O)(=O)CCBr)cc1",
        "O=C1c2ccccc2C(=O)N1/N=N/N1C(=O)c2ccccc2C1=O",
        "O=C([O-])c1ccc(Oc2ccccc2)cc1",
        "CCOC(=O)C1CCc2ccccc2CC1",
        "O=c1cc(-c2ccc(O)cc2)oc2ccccc12",
        "CCc1nnn(-c2ccccc2)c1C",
        "O=C(Cl)c1ccc(C(F)(F)F)cc1"
    ],
    [
        "CN[C@H](C#N)c1ccc(OC)cc1",
        "O=C(Oc1ccccc1)c1cc2ccccc2oc1=O",
        "CC(=O)N[C@H](CS(=O)(=O)Cc1ccc([N+](=O)[O-])cc1)C(=O)[O-]",
        "CC(=S)NC(=O)NC(=O)c1ccc(C)cc1",
        "Nc1nccc2snnc12",
        "CCOC(=O)/C(C#N)=C(\\C)c1ccccc1",
        "O=C1C2=C(C(=O)c3ncccc31)[C@H]1C=C[C@H]2CC1",
        "COC(Cn1cnc2c(N)ncnc21)OC",
        "NC(=O)N(CCc1ccccc1)N=O",
        "COc1cc(C)c(Cl)c(C)c1C(C)=O"
    ],
    [
        "N#C[C@@H](OC(=O)c1ccccc1)c1cccc([N+](=O)[O-])c1",
        "O=C1CS[C@H](c2cc([N+](=O)[O-])ccc2Cl)N1c1nncs1",
        "O=C([O-])c1cccc(N2C(=O)CSC2=S)c1",
        "CCOc1ccc(C2[N@]3CCC[N@H+]2CC3)cc1OC",
        "CC(C)[C@@H](NC(=O)c1cnc2ccccc2n1)C(=O)N[C@@H](C)C(=O)[O-]",
        "CCN(O)CC",
        "O=c1c2ccccc2sc2c1c(N(CCO)CCO)cc1ccccc12",
        "Cc1nc2[nH]nc(N)c2c(C)c1Br",
        "COc1cc(O)c2c(c1)C(=O)c1cc(C)cc(O)c1C2=O",
        "O=C(Nc1ccc(S(=O)(=O)[O-])cc1)c1ccco1"
    ],
    [
        "Cc1ccc(-n2cnc3c(Cl)ncnc32)cc1[N+](=O)[O-]",
        "COc1ccc([C@]2(C(OC)OC)NC(=O)N(C)C2=O)cc1",
        "CC(C)C(=O)C(C)(C)C(=O)N1CCCC1",
        "CCOC(=O)c1ccc(=O)n(Cc2ccccc2)c1",
        "CC(C)(O)[C@@H]1CCCC[C@H]1Cl",
        "Cn1c(=O)nc2n(-c3ccc(Cl)cc3)c3ccc(Cl)cc3nc-2c1=O",
        "c1ccc2c(C[NH+]3CCCCC3)cccc2c1",
        "O=C(CCCc1ccncc1)c1ccccc1",
        "O=C([O-])CNC(=O)CCl",
        "O=C(/C=C/c1ccccc1)C1=C([O-])C(=O)O[C@H]1c1ccccc1"
    ],
    [
        "CC(=O)Nc1ccc(CO[C@H](O)C[NH+]2CCCCC2)cc1[N+](=O)[O-]",
        "NC(=O)COc1c(Cl)cc(Cl)cc1Cl",
        "CN1C(=O)/C(=C2\\Nc3ccccc3C2=O)c2ccccc21",
        "ClCC(CCl)(CCl)CCl",
        "Cc1ccc([S@@](=O)c2ccc([N+](=O)[O-])cc2[N+](=O)[O-])cc1[N+](=O)[O-]",
        "C[C@H]1Oc2c3c(cc(O)c2C(=O)[C@@H]1C)OC(C)(C)CC3",
        "COc1ccc(CC[NH2+]C2=CC(=O)CC2)cc1OC",
        "C[NH+](CCCl)C[C@@H]1CSCO1",
        "O=C1NC(=O)/C(=C/c2ccc(Cl)cc2Cl)N1",
        "C/N=N\\C(C)C"
    ],
    [
        "N#Cc1sc(=S)n(-c2ccccc2)c1N",
        "C[C@@H](NC(=O)Oc1ccccc1)Oc1ccccc1",
        "Nc1ncnc2c1ncn2Cc1cccnc1",
        "C=Cc1c(C)cc(C)cc1C",
        "CN(C)c1nc(Oc2nc(N(C)C)nc(N(C)C)n2)nc(N(C)C)n1",
        "Cc1nc(NC(=O)Nc2ccccc2)sc1C(=O)NN",
        "O=C(N[C@H]1CCC[C@@H]1O)c1ccccc1",
        "CN(N=O)C(=O)NCCOC(=O)N(C)N=O",
        "C[NH+](C)Cc1cc(Cl)ccc1O",
        "O=NN(CC(=O)[O-])Cc1ccccc1"
    ],
    [
        "O=C1c2ccccc2CCc2ccccc21",
        "CN(C)c1ccc(/[N+]([O-])=C/c2cc(=O)c(OS(C)(=O)=O)co2)cc1",
        "COc1cccc2oc([N+](=O)[O-])cc12",
        "Cn1cnc(C(N)=S)c1N",
        "C[NH2+]C1(c2ccc(F)cc2)CCCCC1",
        "O=C([O-])c1cc(I)cc(I)c1",
        "O=C(c1ccc([N+](=O)[O-])cc1)c1ccc(O)cc1O",
        "Cc1cccc(Cl)c1OCC(N)=O",
        "COc1ccc(S(=O)(=O)N(CC(=O)[O-])c2ccccc2)cc1",
        "CCO[C@H]([C@@H](OCC)[n+]1ccccc1)[n+]1ccccc1"
    ],
    [
        "CCOC(=O)[C@H]1C[C@@H]1C(=O)OCC",
        "O=[N+]([O-])c1ccc(N2CCCC2)cc1",
        "CN(C)C(=O)OCCOc1ccc(Cl)cc1",
        "O=C(OCCO)N1CC[NH+](CCO)CC1",
        "S=C(N/[NH+]=C(\\c1ccccc1)c1ccccn1)SCc1ccccc1",
        "O=C([O-])C(=Cc1ccc([N+](=O)[O-])cc1)C(=O)[O-]",
        "C/C(=[NH+]/C(C)(C)C)N1CCCCC1",
        "C[C@H](COC(=O)c1ccccc1)[NH2+]C1CCCCC1",
        "NC(=O)N[C@H]1C[C@@H]1c1ccc(Cl)c(Cl)c1",
        "N#CC(C#N)C1c2ccccc2-c2ccccc21"
    ],
    [
        "CCOC(=O)[C@H]1C[C@H](O)C[NH2+]1",
        "Cc1ncc(C(=O)[O-])s1",
        "CN1C(=O)N(C)C(=O)C(Nc2ccc([N+](=O)[O-])cc2)(SCCO)C1=O",
        "O=NN(CCCl)C(=O)Nc1ccc(CC(=O)[O-])cc1",
        "CN1COc2ccc(Br)cc2C1",
        "C[N+](C)(C)CC(=O)N/[NH+]=C/c1ccc([N+](=O)[O-])o1",
        "COc1cc2ccccc2[n+](C)c1-c1ccccc1F",
        "Nc1nc(N)c2c(ccc3ccc(Cl)cc32)n1",
        "[O-][n+]1cc[n+]([O-])c2c3ccccc3ccc21",
        "O=C(NN1CCCCC1)c1ccccc1"
    ],
    [
        "CCOC(=O)/C=C1\\Nc2ccccc2C(c2ccccc2)=[NH+][C@H]1OC(C)=O",
        "C/C(CCO)=[NH+]\\NC(N)=S",
        "Cc1c(C)c(C)c(C[NH+]2CCCC2)c(C)c1C",
        "C[S+](C)CCO",
        "O=C1N=C(N2CCN(c3ccccc3)CC2)N[C@@H]2CCCC[C@H]12",
        "Cc1[nH]c(N)nc(=O)c1CCCNC(=O)c1ccc(N)cc1",
        "CCCCc1ccc([I+]c2cccc([N+](=O)[O-])c2)cc1",
        "CCN(CC)CC#CCN(CC)CC",
        "C[C@H](Oc1ccc2cc(C(=O)c3ccccc3)c(=O)oc2c1)C(=O)[O-]",
        "COC(=O)c1ccccc1NC(=S)NC(=O)COc1ccccc1"
    ],
    [
        "COc1ccc(OC)c(/C=C/C=O)c1",
        "CCCCN(CC)CC#N",
        "N#Cc1cccc(F)c1",
        "Cc1cc2ccccc2nc1Cl",
        "CN(C=O)COCN(C)C=O",
        "Cc1ccc(N2CC[NH+](C[C@H](O)COc3ccccc3)CC2)cc1",
        "O=c1oc(-c2ccc(O)c(O)c2)cc2cc(O)cc([O-])c12",
        "COc1ccc(/C=C2/SC(=O)NC2=O)cc1OC",
        "O=S(=O)(NCc1ccco1)c1ccc(F)cc1",
        "O=C([O-])c1ccc(NCn2c(=S)[nH]c3ccccc32)cc1"
    ],
    [
        "O=c1[nH]c(=O)ss1",
        "Cc1[nH]c2ccc(C(=O)NN)cc2c1CC(=O)NN",
        "Cc1ccccc1C(=O)[C@@H]1CCCCC1=O",
        "COc1cc(CCl)cc(OC)c1OC",
        "CC[NH+]1C[C@@H](O)C2=C/C(=[NH+]/NC(N)=O)C(=O)C=C21",
        "COc1ccccc1[C@@H](C)c1ccc2ccccc2c1C(=O)[O-]",
        "O=C(NCCc1cnc[nH]1)C(F)(F)F",
        "C[N+](C)(C)CCOC(=O)c1ccc(CBr)cc1",
        "COc1ccc2c(c1)OC/C(=C\\c1ccc([N+](=O)[O-])o1)C2=O",
        "COc1c(O)ccc2c1C[NH+]1CCc3cc4c(cc3[C@@H]1C2)OCO4"
    ],
    [
        "O=C(NC(=O)c1cc(-c2ccccc2)n[nH]1)c1ccccc1",
        "C(=[NH+]/[NH+]=C/c1cccs1)\\c1cccs1",
        "Cn1c2c(c3ccccc31)CCN(CCCC[NH+]1CCCCC1)C2=O",
        "CC(=O)CC(=O)OC[C@H]1COCO1",
        "CC(=O)N[C@H](C)C(=O)N(C)/N=N/c1ccc(C#N)cc1",
        "CCCCCCOc1ccccc1CO",
        "CC(=O)c1c(COc2ccc(Cl)cc2)[n+]([O-])c2ccccc2[n+]1[O-]",
        "CCOC(=O)CNC(=O)Oc1cccc2cccnc12",
        "O=C1N[C@@H](Cc2ccccc2)C(=O)O1",
        "CCOC(=O)C(=O)NCc1ccc(Cl)cc1Cl"
    ],
    [
        "CC(=O)c1cc(=O)[nH]c(=O)[nH]1",
        "COC(=O)C[C@@]1(C)[C@H](O)[C@H]2OC(C)(C)O[C@H]2[C@@H]1O",
        "c1ccc(/C(=[NH+]\\[NH+]=c2\\[nH]c3ccccc3s2)c2ccncc2)cc1",
        "Cc1cccc(Cl)c1",
        "NC(=S)Nn1c(=O)c2ccccc2c2ccccc2c1=O",
        "O=C1NCc2nnn(Cc3ccccc3)c2N1",
        "Nc1ccc(/N=N/c2ccc3nccc(N)c3c2)c(N)n1",
        "Cc1ccc(NC(=S)C#N)c(C)c1",
        "COC(=O)[C@H](CC=O)c1ccc(OC)cc1",
        "CC(C)(CC(=O)[O-])NC(=O)CN"
    ],
    [
        "CC1(C)CC(=O)c2c(oc(=O)c3ccccc23)C1",
        "CCOC(=O)c1cnc2c(OC)cccc2c1O",
        "OCc1c(CO)c(Br)c2ccccc2c1Br",
        "COc1ccc(C)cc1S(=O)(=O)c1ccc(C)cc1[N+](=O)[O-]",
        "Cc1ccc(C(=O)C[N+]2(C)CCOCC2)cc1",
        "C1CCC[NH+](CCN2CCOCC2)CCC1",
        "Cc1cc(O)c(C(C)C)cc1C[NH+]1CCCCC1",
        "O=C([O-])/C(=C\\c1cccc(Br)c1)c1ccccc1",
        "C=C(C)[C@H]1C[C@@H]2O[C@]2(C)[C@@H](O)C1",
        "O=[N+]([O-])c1cc/c(=[NH+]\\[NH+]=C/c2ccc(O)c(S(=O)(=O)[O-])c2)[nH]c1"
    ]
]

In [35]:
def unique_random_numbers(n):
    rands = []
    i = 0
    while i < n:
        rand = random.randint(0,9)
        if rand in rands:
            continue
        else:
            rands.append(rand)
            i += 1

    return rands

unique_random_numbers(5)

[4, 7, 9, 5, 1]

In [36]:
best = dataset.get_top_or_bottom(1320) # best mols.
new_SMILES = []

for i in range(5):
    temp_new_SMILES = []
    for j in range(30):
        best_samples = best.get_samples(i+1).ids.tolist()
        rands = unique_random_numbers(i+1)
        temp_SMILES = SMILES[j].copy()

        for i, rand in enumerate(rands):
            temp_SMILES[rand] = best_samples[i]
            
        temp_new_SMILES.append(temp_SMILES)
    new_SMILES.append(temp_new_SMILES)

new_SMILES[0]

[['O=C(CCl)NCc1cnccn1',
  'Cn1cnc([N+](=O)[O-])c1SSc1c([N+](=O)[O-])ncn1C',
  'Cc1ccc(S(=O)(=O)N2Cc3ccc([N+](=O)[O-])cc3C2)cc1',
  'NC(=O)/N=N/c1c(F)c(F)c(F)c(F)c1F',
  'CCCOC(=O)NCCSC(=O)OCCC',
  'O=C(NCC1CC1)c1ccccc1NC(=O)c1cc2ccccc2oc1=O',
  'CC[C@H](C)[C@H](N)C(=O)NCC(=O)OC',
  'C/C(=[NH+]\\S(=O)(=O)c1cccc([N+](=O)[O-])c1)OCC(C)C',
  'COc1ccc(/C=N/n2c(-c3ccccc3)n[nH]c2=O)c(OC)c1',
  'CC(=O)Nc1ccc(S(=O)(=O)F)cc1Cl'],
 ['COc1c(O)cc(Br)c(C=O)c1N',
  'CCCCN1C(=O)NC(=O)[C@@]1(C)c1ccccc1',
  'c1ccc(C2=[NH+]N[C@@H]3c4ccccc4-c4ccccc4N3C2)cc1',
  'CCCCCC1(C(C)C)C(=O)NC(=O)NC1=O',
  'c1c2c(cc3c1OCN(C1CC1)C3)OCN(C1CC1)C2',
  'CC[C@@H](O)[C@H]1Cc2c(C)cc3c(c2O1)C(=O)c1ccccc1C3=O',
  'COc1ccc2cc(N3CCCC3)ccc2c1',
  'O=C(CSc1ncc[nH]1)c1ccc(Br)cc1',
  'C/C=C/[C@H](C(=O)NC(N)=O)C(C)C',
  'C=CCNC(=S)NCc1cccc2ccccc12'],
 ['CCN(CC)/[NH+]=C/C[C@H](C)Nc1ccnc2cc(Cl)ccc12',
  'COc1ccc2nc(C)c(COC(C)=O)cc2c1',
  'C[NH+]1C=c2c3c(ccc2=C2C=C[C@H]4C=C5OCOC5=CC4=C21)OCO3',
  'O=C([O-])c1cccc(O)c1O',
  'O=CNc1cc(B

In [37]:
dataset.get_points_from_ids(['COc1ccc2cc(C(=O)N3CCC(O)(Cc4ccncc4)CC3)c(C)nc2c1']).y

array([], dtype=float64)

In [38]:
new_SMILES

[[['O=C(CCl)NCc1cnccn1',
   'Cn1cnc([N+](=O)[O-])c1SSc1c([N+](=O)[O-])ncn1C',
   'Cc1ccc(S(=O)(=O)N2Cc3ccc([N+](=O)[O-])cc3C2)cc1',
   'NC(=O)/N=N/c1c(F)c(F)c(F)c(F)c1F',
   'CCCOC(=O)NCCSC(=O)OCCC',
   'O=C(NCC1CC1)c1ccccc1NC(=O)c1cc2ccccc2oc1=O',
   'CC[C@H](C)[C@H](N)C(=O)NCC(=O)OC',
   'C/C(=[NH+]\\S(=O)(=O)c1cccc([N+](=O)[O-])c1)OCC(C)C',
   'COc1ccc(/C=N/n2c(-c3ccccc3)n[nH]c2=O)c(OC)c1',
   'CC(=O)Nc1ccc(S(=O)(=O)F)cc1Cl'],
  ['COc1c(O)cc(Br)c(C=O)c1N',
   'CCCCN1C(=O)NC(=O)[C@@]1(C)c1ccccc1',
   'c1ccc(C2=[NH+]N[C@@H]3c4ccccc4-c4ccccc4N3C2)cc1',
   'CCCCCC1(C(C)C)C(=O)NC(=O)NC1=O',
   'c1c2c(cc3c1OCN(C1CC1)C3)OCN(C1CC1)C2',
   'CC[C@@H](O)[C@H]1Cc2c(C)cc3c(c2O1)C(=O)c1ccccc1C3=O',
   'COc1ccc2cc(N3CCCC3)ccc2c1',
   'O=C(CSc1ncc[nH]1)c1ccc(Br)cc1',
   'C/C=C/[C@H](C(=O)NC(N)=O)C(C)C',
   'C=CCNC(=S)NCc1cccc2ccccc12'],
  ['CCN(CC)/[NH+]=C/C[C@H](C)Nc1ccnc2cc(Cl)ccc12',
   'COc1ccc2nc(C)c(COC(C)=O)cc2c1',
   'C[NH+]1C=c2c3c(ccc2=C2C=C[C@H]4C=C5OCOC5=CC4=C21)OCO3',
   'O=C([O-])c1cc

In [39]:
len(new_SMILES)

5

In [40]:
# Tests list for number of unique molecules compared the list without enriched molecules.

count = 0
# Go through every enriched list of lists.
for nudged_lists in new_SMILES:
    
    # go through the 30 enriched lists. To compare them with the normal list we tuple them together with the non-enriched lists.
    ok_count = 0
    for unique_list, normal_list in zip(nudged_lists, SMILES):
        
        # if there is a unique molecule in a list add +1 to `unique_count`
        unique_count = 0
        for i in normal_list:
            if i in unique_list:
                pass
            else:
                unique_count += 1
        
        # a list have the correct number of unique molecules if it corresponds to count + 1.
        # That is, the first list of lists should have 1 unique aka count(0) + 1
        # the second list should have 2 unique and so on.
        if unique_count == (count + 1):
            ok_count += 1

    # For all list of lists to have the correct number of uniques, ok_count will equal 30.
    if ok_count == 30:
        print(f"All list {count+1} have {unique_count} unique(s)")
                
    count += 1

All list 1 have 1 unique(s)
All list 2 have 2 unique(s)
All list 3 have 3 unique(s)
All list 4 have 4 unique(s)
All list 5 have 5 unique(s)


In [41]:
# Test that each list contains 10 items.
count_nudged = 0
for nudged_lists in new_SMILES:
    
    count_unique = 0
    for unique_list in nudged_lists:
        if len(unique_list) != 10:
            print(f"Problem with nudge list {count_nudged} and unique list {count_unique}")
        count_unique += 1
    
    count_nudged += 1
print("Finished counting.")

Finished counting.


In [42]:
# Test that every item in every list is unique

missing_smiles = [smile for smile in temp_SMILES if smile not in dataset.ids]

for nudged_lists in new_SMILES:
    for unique_list in nudged_lists:
        duplicates = [smile for smile in unique_list if unique_list.count(smile) > 1]
        if duplicates:
            print("Duplicate SMILES found:", duplicates)
        else:
            pass

In [43]:
dataset_best_sorted = Dataset.load('../datasets/DTP_MQN_shuffled.pkl')
dataset_best_sorted.sort_by_y()

# Ensure every that every SMILES in the original list can be found in the imported dataset.
list = SMILES[10]
for list in SMILES:
    if len(dataset_best_sorted.get_points_from_ids(list).y) != 10:
        print("You done messed up?")

In [44]:
# Test that each newly generated list have 10 items.

for nudged_lists in new_SMILES:
    for unique_list in nudged_lists:
        if len(unique_list) != 10:
            print("You done messed up?")

In [48]:
# Test that every SMILES exists in the dataset.

for nudged_lists in new_SMILES:
    for list in nudged_lists:
        if len(dataset.get_points_from_ids(list).y) != 10:
            print("You done messed up?")

In [51]:
new_SMILES[4]

[['O=C(CCl)NCc1cnccn1',
  'OC[C@H]1C[C@](O)(c2ccc(F)cc2)[C@@H](c2ccc(F)cc2)O1',
  'Cc1ccc(S(=O)(=O)N2Cc3ccc([N+](=O)[O-])cc3C2)cc1',
  'CC(=O)Nc1ccc(S(=O)(=O)[C@H](Cl)c2ccccc2)cc1',
  'CCCOC(=O)NCCSC(=O)OCCC',
  'CC(=O)NC(CCC(=O)c1ccc(Cl)cc1)(C(=O)[O-])C(=O)[O-]',
  'C[C@@H](O)COc1ccccc1P(c1ccccc1)c1ccccc1',
  'CCOc1cc([C@H](O)[C@@H](O)c2ccc(O)c(OCC)c2)ccc1O',
  'COc1ccccc1[C@@H](O)c1cnc(C)n1Cc1ccccc1',
  'CC(=O)Nc1ccc(S(=O)(=O)F)cc1Cl'],
 ['COc1c(O)cc(Br)c(C=O)c1N',
  'CCCCN1C(=O)NC(=O)[C@@]1(C)c1ccccc1',
  'Nc1cccc(-c2nc3ccc4c(O)cc(S(=O)(=O)[O-])cc4c3s2)c1',
  'COC(=O)[C@@]1(Cc2ccc3c(c2)CCC3)Cc2ccccc2[C@H]1O',
  'c1c2c(cc3c1OCN(C1CC1)C3)OCN(C1CC1)C2',
  'CCCCc1cc(O)ccc1O',
  'CCOc1cc(CC[NH2+]C)c2c(ccc3ccc(OC)c(O)c32)c1O',
  'O=C(CSc1ncc[nH]1)c1ccc(Br)cc1',
  'CC[NH+](CC)Cc1cc2c(cc1O)CC[C@@H]1[C@@H]2CC[C@]2(C)C(=O)CC[C@@H]12',
  'Cc1ccc(C(=O)C[C@H](c2ccc(C)cc2)C2C(=O)NC(=O)NC2=O)cc1'],
 ['CCN(CC)/[NH+]=C/C[C@H](C)Nc1ccnc2cc(Cl)ccc12',
  'COc1ccc2nc(C)c(COC(C)=O)cc2c1',
  'Oc1ccc([C@H]